## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vardo,NO,70.3705,31.1107,24.78,93,100,34.52,fog
1,1,Thaton,MM,16.9206,97.3714,82.17,77,24,1.45,few clouds
2,2,Wundanyi,KE,-3.4019,38.3640,62.29,89,94,4.74,overcast clouds
3,3,Khatanga,RU,71.9667,102.5000,13.30,84,100,10.25,overcast clouds
4,4,Tasiilaq,GL,65.6145,-37.6368,33.96,79,99,5.35,overcast clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Atuona,PF,-9.8000,-139.0333,78.62,77,29,21.90,light rain
14,14,Rikitea,PF,-23.1203,-134.9692,77.16,72,7,24.67,clear sky
21,21,Port Hedland,AU,-20.3167,118.5667,77.05,78,40,0.00,scattered clouds
22,22,Santa Clarita,US,34.3917,-118.5426,79.84,44,0,11.50,clear sky
24,24,Vaini,TO,-21.2000,-175.2000,77.16,61,20,9.22,few clouds
25,25,Cidreira,BR,-30.1811,-50.2056,79.61,73,10,8.84,clear sky
31,31,Kroya,ID,-7.6331,109.2461,78.91,85,100,1.32,overcast clouds
33,33,Antsohihy,MG,-14.8796,47.9875,76.50,85,31,2.84,scattered clouds
34,34,Auki,SB,-8.7676,160.7034,77.49,85,89,3.20,overcast clouds
36,36,Padang,ID,-0.9492,100.3543,78.64,87,96,2.30,light rain


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                80
City                   80
Country                80
Lat                    80
Lng                    80
Max Temp               80
Humidity               80
Cloudiness             80
Wind Speed             80
Current Description    80
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Atuona,PF,78.62,light rain,-9.8000,-139.0333,
14,Rikitea,PF,77.16,clear sky,-23.1203,-134.9692,
21,Port Hedland,AU,77.05,scattered clouds,-20.3167,118.5667,
22,Santa Clarita,US,79.84,clear sky,34.3917,-118.5426,
24,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
25,Cidreira,BR,79.61,clear sky,-30.1811,-50.2056,
31,Kroya,ID,78.91,overcast clouds,-7.6331,109.2461,
33,Antsohihy,MG,76.50,scattered clouds,-14.8796,47.9875,
34,Auki,SB,77.49,overcast clouds,-8.7676,160.7034,
36,Padang,ID,78.64,light rain,-0.9492,100.3543,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary. 
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace(" ", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Atuona,PF,78.62,light rain,-9.8000,-139.0333,Villa Enata
14,Rikitea,PF,77.16,clear sky,-23.1203,-134.9692,People ThankYou
21,Port Hedland,AU,77.05,scattered clouds,-20.3167,118.5667,The Esplanade Hotel
22,Santa Clarita,US,79.84,clear sky,34.3917,-118.5426,Hyatt Regency Valencia
24,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))